## Syntax-dependent trace

Head attribution

In [10]:
## Jupyter setup
%load_ext autoreload
%autoreload 2

# %env CUDA_VISIBLE_DEVICES=0
# !echo $CUDA_VISIBLE_DEVICES

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import os

## discovery cluster setup
# CACHE_DIR = "/scratch/lucchetti.f/models/"
# os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR

## coreweave cluster setup
CACHE_DIR = "/data/home/franlucc"
os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR

In [3]:
import sys
sys.path.append('..')

import numpy as np
import gc
import torch
from model_utils import *
import tqdm as notebook_tqdm
from trace_model import ModelLoader
torch.set_grad_enabled(False)
import circuitsvis as cv
from IPython.display import display

#MODEL_NAME = "Salesforce/codegen-16B-mono"
MODEL_NAME = "bigcode/starcoder"
# MODEL_NAME = "bigcode/santacoder"

check_devs()
torch.cuda.is_available()
clear_devs()

0 / 84979089408 used for device 0, reserved 0


In [4]:
lm = ModelLoader(MODEL_NAME, AUTH="AUTH_TOK_HERE")
check_devs()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

62136934400 / 84979089408 used for device 0, reserved 62138613760


In [5]:
check_devs()

62136934400 / 84979089408 used for device 0, reserved 62138613760


## Test Trace


In [12]:
prompts = ["""
        int x = 10;
        int y = 0;"""]


txt, ret_dict = lm.trace_generate(
    prompts,
    max_out_len=  len(prompts[0]) + 5,
    argmax_greedy= False,
    debug = False,
    request_activations= [lm.layer_name_format.format(i) for i in range(0,40)],
    use_cache=False,
    top_k = 20,
)
txt, ret_dict

/data/home/franlucc/codetrace/notebooks/../trace_model.py:132: UserWarning: The model `bigcode/starcoder` of type `gpt_bigcode` already implements or can't utilize `use_cache` for fast generation. Setting `use_cache = False`.
  warnings.warn(f"The model `{self.model_name}` of type `{self.model_type}` already implements or can't utilize `use_cache` for fast generation. Setting `use_cache = False`.")


(["\n        int x = 10;\n        int y = 0;\n        int z = x + y;\n      }\n      ''')\n\n    # Indirect call to a Java-like method.\n    java_test"],
 {'input_tokenized': [[('\n       ', 291),
    (' int', 629),
    (' x', 816),
    (' =', 280),
    (' ', 225),
    ('1', 35),
    ('0', 34),
    (';', 45),
    ('\n       ', 291),
    (' int', 629),
    (' y', 533),
    (' =', 280),
    (' ', 225),
    ('0', 34),
    (';', 45)]],
  'generated_tokens': [[[{'token': '\n       ',
      'id': 291,
      'p': 0.8093909025192261},
     {'token': '\n\n       ', 'id': 584, 'p': 0.10156369209289551},
     {'token': '\n   ', 'id': 284, 'p': 0.03489241749048233},
     {'token': '\n        \n       ', 'id': 2746, 'p': 0.01691375859081745},
     {'token': ' //', 'id': 434, 'p': 0.0070964135229587555},
     {'token': '\n', 'id': 203, 'p': 0.00511282728984952},
     {'token': '\n     ', 'id': 397, 'p': 0.0028335305396467447},
     {'token': ' \n       ', 'id': 4456, 'p': 0.00249982881359756},
     

## Logit lens

In [6]:
lm.get_logits(['int x = '])


--- Argument Model Logit Lens ---
0: [(' ', 1), ('<fim_suffix>', 0), ('\n', 0), (' (', 0), ('1', 0)]
1: [(' ', 1), ('<fim_suffix>', 0), (' (', 0), ('1', 0), ('\n', 0)]
2: [(' ', 1), ('1', 0), ('<fim_suffix>', 0), ('\n', 0), (' (', 0)]
3: [(' ', 1), ('1', 1), ('\n', 1), (' (', 1), ('<fim_suffix>', 0)]
4: [(' ', 1), ('1', 1), (' (', 1), ('\n', 1), ('0', 0)]
5: [('1', 3), (' (', 2), (' ', 2), ('0', 1), ('\n', 1)]
6: [('1', 5), (' (', 3), ('0', 3), (' ', 3), ('\n', 1)]
7: [('1', 5), (' ', 3), ('0', 3), (' (', 2), (' -', 1)]
8: [(' ', 5), ('1', 5), ('0', 4), (' (', 3), ('<fim_middle>', 1)]
9: [('0', 4), (' ', 4), ('1', 4), (' (', 3), ('2', 1)]
10: [(' ', 7), (' (', 7), ('0', 4), ('1', 3), (' -', 1)]
11: [(' ', 11), (' (', 9), ('0', 5), ('1', 4), (' -', 1)]
12: [('0', 10), (' ', 9), (' (', 8), ('1', 7), (' -', 2)]
13: [(' ', 11), ('0', 10), (' (', 8), ('1', 6), (' -', 2)]
14: [(' ', 16), ('0', 11), (' (', 6), ('1', 6), (' -', 2)]
15: [('0', 21), (' ', 13), ('1', 10), (' (', 4), ('2', 2)]
16

## Attention visualize

Using circuitvis

[nhead, seq_dest, seq_src]

In [18]:
layer = 19
attention = ret_dict["activations"][f"transformer.h.{layer}"][0]
tokens = list(list(zip(*ret_dict["input_tokenized"][0]))[0])
print(tokens)

['\n       ', ' int', ' x', ' =', ' ', '1', '0', ';', '\n       ', ' int', ' y', ' =', ' ', '0', ';']


In [19]:
display(cv.attention.attention_patterns(tokens=tokens, attention=attention))

In [44]:
def induction_attn_detector(ret_dict, n_layers=40, n_heads=48) -> List[str]:
    '''
    Returns a list e.g. ["0.2", "1.4", "1.9"] of "layer.head" which you judge to be induction heads

    Remember - the tokens used to generate rep_cache are (bos_token, *rand_tokens, *rand_tokens)
    '''
    attn_heads = []
    for layer in range(n_layers):
        for head in range(n_heads):
            attention_pattern = ret_dict["activations"][f"transformer.h.{layer}"][0][head]
            # take avg of (-seq_len+1)-offset elements
            seq_len = (attention_pattern.shape[-1] - 1) // 2
            score = attention_pattern.diagonal(-seq_len+1).mean()
            if score > 0.06: ## arb threshold
                attn_heads.append(f"{layer}.{head}")
    return attn_heads


print("Induction heads = ", "\n".join(induction_attn_detector(ret_dict)))

Induction heads =  19.13
31.2
